# MF를 이용한 추천시스템

### MF (Matrix Factorization, 행렬분해) : 하나의 행렬을 두 개이상의 행렬의 곱과 같도록 행렬을 분해하는 것을 의미
- 보통 행렬 분해 한 후 분해된 행렬의 일부로 원본 행렬을 추정하도록 사용
- 분해 시 차원을 낮추기 때문에 낮아진 차원의 요소들은 원래 행렬의 핵심 요소를 설명할 수 있다. 또한 희박한 데이터 세트에서도 잘 동작합니다.
    - 연관된 알고리즘 : PCA, SVD, MF
        - 사용자 요인 $P$
        - ITEM 요인 $Q$
        - $R = P * Q^T$
    

## Env

In [18]:
import numpy as np
import pandas as pd

In [19]:
P = np.array([[-0.43, 0.21],[0.31, 0.92],[0.69,-0.03],[0.46,-0.30]])
Q = np.array([[0.31, 0.60],[0.61, -0.82],[-0.38,-0.61],[-0.79,0.08]])

In [20]:
P

array([[-0.43,  0.21],
       [ 0.31,  0.92],
       [ 0.69, -0.03],
       [ 0.46, -0.3 ]])

In [21]:
Q

array([[ 0.31,  0.6 ],
       [ 0.61, -0.82],
       [-0.38, -0.61],
       [-0.79,  0.08]])

In [22]:
R = P.dot(Q.T)
R

array([[-0.0073, -0.4345,  0.0353,  0.3565],
       [ 0.6481, -0.5653, -0.679 , -0.1713],
       [ 0.1959,  0.4455, -0.2439, -0.5475],
       [-0.0374,  0.5266,  0.0082, -0.3874]])

## Data load

In [23]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.user', sep='|', names=u_cols, encoding='latin-1')

In [24]:
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown',
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary',
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
          'Thriller', 'War', 'Western']
movies = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [25]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [26]:
# timestamp 제거
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


## MF Class
- 결손값을 메꾸지 않고 관측된 평가값만을 사용하여 latent factor model 구현
- 기계학습 방법을 이용하여 최대한 $R$ 에 가까운 근사값 $\hat{R}$ 을 구함
    - $R$ = $|U| * |I|$ : user-item rating matrix ($rank k < n$)
    - $P^T \rightarrow |U| * k$ : user latent matrix
    - $Q \rightarrow k * |I|$ : item latent matrix
    - $R \approx P^T * Q = \hat{R}$
    
- Optimization 방법
    - SGD (Stochastic Gradient Descent)
    - ALS (Alternating Least Squares)

In [27]:
class MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = np.array(ratings)
        self.num_users, self.num_items = np.shape(self.R) # 행과 열 개수를 반환하여 사용자 수와 아이템 수를 저장
        self.K = K            # 잠재 요인의 차원
        self.alpha = alpha    # 학습률
        self.beta = beta      # 정규화 계수 (오버피팅 방지)
        self.iterations = iterations  # 반복횟수
        self.verbose = verbose # 학습 과정 출력 여부

    # Root Mean Squared Error (RMSE) 계산 
    def rmse(self):
        xs, ys = self.R.nonzero()     # R에서 평점이 있는 요소의 index (0이 아닌 값의 인덱스)
        self.predictions = [] # 예측 평가값
        self.errors = [] 
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y) 
            self.predictions.append(prediction) # 사용자 x와 아이템 y의 예측 평점을 반환
            self.errors.append(self.R[x, y] - prediction) # 모든 X,y에 대해 처리
        self.predictions = np.array(self.predictions) # 예측된 평점 리스트
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2)) # 평균 제곱 오차의 제곱근

    # 정해진 횟수만큼 P, Q, bu, bd 값을 update
    def train(self): # 편미분
        
        # Initializing user-feature and item-feature matrix
        # 평균 0, 표준편차 1/k인 정규분포를 갖는 난수로 초기화 
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K)) 
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()]) # 관측된 평점의 평균값

        # List of training samples
        rows, columns = self.R.nonzero() 
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]

        # 주어진 반복 횟수에 대해 Stochastic gradient descent 
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            training_process.append((i+1, rmse))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f " % (i+1, rmse))
        return training_process

    # 사용자 i 및 항목 j에 대한 등급 평점 예측
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Stochastic gradient descent 를 사용하여 최적화된 P 및 Q 행렬을 얻는다.
    def sgd(self): # 경사하강법
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

1. 클래스 선언과 초기화
2. RMSE 계산
    - `self.R.nonzero()` : self.R에서 0이 아닌 값의 인덱스를 반환
    - `get_predictions(x, y)`: 사용자 x와 아이템 y의 예측 평점을 반환
    - `self.predictions`: 예측된 평점 리스트
    - `self.errors`: 실제 평점과 예측 평점의 차이를 저장한 리스트
3. 학습
4. 평점 예측
    - $ \hat{r}_{ij} = b + b_u[i] + b_d[j] + P[i] \cdot Q[j]^T $
    - $b$ : 전반적인 바이어스
    - $b_u[i]$ : 사용자 i의 바이어스
    - $b_d[j]$ : 아이템 j의 바이어스
    - $P[i] \cdot Q[j]^T$ : 사용자와 아이템 간의 latent factor 내적
5. SGD 구현
    - 오차 계산 : $e = r - \hat{r}$ : 실제 평점과 예측 평점의 차이
    - 바이어스 업데이트 : $b_u[i]$ 와 $b_d[j]$ : 사용자와 아이템 바이어스 업데이트
    - 잠재 요인 업데이트 : $P[i]$ 와 $Q[j]$ : 사용자와 아이템의 잠재 요인 업데이트

- SGD를 통해  $P ,  Q ,  b_u ,  b_d$ 를 최적화한다. 
- 학습 과정을 거처 새로운 사용자 - 아이템 조합에 대한 평점을 예측가능


# 전체 데이터에서 추천

In [28]:
R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
mf = MF(R_temp, K=30, alpha=0.001, beta=0.02, iterations=150, verbose=True)
train_process = mf.train()

Iteration: 10 ; Train RMSE = 0.9585 
Iteration: 20 ; Train RMSE = 0.9374 
Iteration: 30 ; Train RMSE = 0.9281 
Iteration: 40 ; Train RMSE = 0.9225 
Iteration: 50 ; Train RMSE = 0.9184 
Iteration: 60 ; Train RMSE = 0.9146 
Iteration: 70 ; Train RMSE = 0.9102 
Iteration: 80 ; Train RMSE = 0.9041 
Iteration: 90 ; Train RMSE = 0.8956 
Iteration: 100 ; Train RMSE = 0.8840 
Iteration: 110 ; Train RMSE = 0.8700 
Iteration: 120 ; Train RMSE = 0.8544 
Iteration: 130 ; Train RMSE = 0.8375 
Iteration: 140 ; Train RMSE = 0.8196 
Iteration: 150 ; Train RMSE = 0.8010 
